In [2]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from pyspark.sql.functions import * 
import zipfile
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient

In [4]:
file_path='CENSUS/supp folder file and codebook path.csv'

In [5]:
# Azure storage access info 
blob_account_name = 'usafactsbronze' # replace with your blob name 
blob_container_name = 'ingestion-meta' # replace with your container name  
linked_service_name = 'bronze' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
# Allow SPARK to access from Blob remotely 
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, file_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

url_df = spark.read.csv(wasbs_path,header=True)

In [6]:
def read_df(blob_path):
    # Azure storage access info 
    blob_account_name = 'usafactsbronze' # replace with your blob name 
    blob_container_name = 'bronze' # replace with your container name  
    linked_service_name = 'bronze' # replace with your linked service name 

    blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
    # Allow SPARK to access from Blob remotely 
    wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_path) 
    spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 
    return wasbs_path

creating a list of tuples that will be used to split the columns of a dataset based on their index positions

In [8]:
def tuples_list(code_book_df):
    df_no_duplicates =code_book_df.dropDuplicates(["Start", "Size","Name"]) 
    selected_columns = ["Start", "Size","Name"]
    list_of_tuples = df_no_duplicates.select(selected_columns).rdd.map(tuple).collect()
    split_columns=[]
    for Tape_start,length,Field_name in list_of_tuples :

        column_name = 'value'

        split_columns.append(expr(f"substring({column_name}, {Tape_start},{length})").alias(Field_name))
    return split_columns

In [9]:
# silver storage access
# Azure storage access info 
blob_account_name = 'usafactssilver'
blob_container_name = 'silver'
linked_service_name = 'silver' 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 


In [10]:
for file_path,codebook_path in url_df.rdd.collect():
    txt_file_blob_path = file_path
    code_book_blob_path = codebook_path
    file_name=txt_file_blob_path.split("/")[-1].split(".")[0]
    folder_path='/'.join(txt_file_blob_path.split("/")[:-2])

    output_blob_path1= f'{folder_path}/{file_name}'

    input_txt_file_df = spark.read.text(read_df(txt_file_blob_path))

    code_book_df=spark.read.option("multiLine","true").csv(read_df(code_book_blob_path),header=True)

    code_book_df = code_book_df.withColumn("Name", regexp_replace(col("Name"), " ", "_"))
    code_book_df=code_book_df.dropDuplicates()
    code_book_df =code_book_df.dropDuplicates(["Name","Code"])

    input_txt_file_df = input_txt_file_df.select("*",*tuples_list(code_book_df)).drop('value')

    # Allow SPARK to access from Blob remotely
    wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, output_blob_path1)
    spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
    
    input_txt_file_df.write.format('delta').mode('overwrite').option("overwriteSchema",True).option("path",wasbs_path).save()
    print('file_written: ', output_blob_path1)
